# Introduction

Copyright 2022 Moran Innovation LLC

   Licensed under the Apache License, Version 2.0 (the "License");
   you may not use this file except in compliance with the License.
   You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0

   Unless required by applicable law or agreed to in writing, software
   distributed under the License is distributed on an "AS IS" BASIS,
   WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
   See the License for the specific language governing permissions and
   limitations under the License.

CryoFM(TM) is a trademark of Moran Innovation LLC. All rights reserved.

**HOW TO USE THIS INTERACTIVE NOTEBOOK: Use the table of contents icon and sidebar at the left to navigate to the topic of interest. Change any of the input values and run the cell code.**

**CAUTION: None of the code cells run automatically. The user must manually run selected code cells in sequence to achieve the desired results. Default units are SI.**

**The reference report for this interactive notebook can be found at: https://drive.google.com/file/d/1sTNNPRgGdC4JrDt5UGz7pkyhoysBNkRZ/view.** 

**Please report any errata or other feedback to info@moraninnovation.com.**

## CryoFM

Try using class for dimensionless numbers?

In [1]:
import CryoFM

help(CryoFM)
help(CryoFM.list_fluids)
help(CryoFM.reynolds)

Help on module CryoFM:

NAME
    CryoFM

DESCRIPTION
    CryoFM(TM) is a library of functions useful for cryogenic fluid management. 
    Default units are SI. Copyright 2022 Moran Innovation LLC. Licensed under 
    the Apache License, Version 2.0. The reference report can be accessed at: 
    https://drive.google.com/file/d/1sTNNPRgGdC4JrDt5UGz7pkyhoysBNkRZ/view

FUNCTIONS
    bond(fluid, accel, diam_fs, press)
        # Bond number based on fluid, acceleration (m/s^2), free surface diameter (m), 
        # and pressure (Pa)
    
    list_fluids()
        # List of commmon cryogenic fluids supported in CoolProp; full list at
        # http://coolprop.org/fluid_properties/PurePseudoPure.html#list-of-fluids
    
    reynolds(fluid, velocity, diam, temp, press)
        # Reynolds number based on fluid, velocity (m/s), characteristic length or
        # hydraulic diameter = 4*area/wetted perimeter (m), fluid temperature (K), and
        # fluid pressure (Pa)

FILE
    c:\users\moran\oned

In [16]:
import CryoFM

list_fluids = CryoFM.list_fluids()
print(list_fluids)

bond_number = CryoFM.bond('Oxygen', 9.81e-6, 6.6, 2e5)
print("Bond number =", round(bond_number,1))

reynolds_number = CryoFM.reynolds('Oxygen', 0.1, 10, 97, 2e5)
print("Reynolds number =", "{:.5e}".format(reynolds_number))

['Common cryogenic fluids:', 'ParaHydrogen', 'Hydrogen', 'OrthoHydrogen', 'Oxygen', 'Methane', 'Nitrogen', 'Helium', 'Neon']
Bond number = 41.1
Reynolds number = 6.75232e+06


## Fluid Properties

[Link](https://github.com/moranmatthewe/CryoFM/blob/main/Saturation%20Conditions%20for%20H2_O2_CH4.jpg?raw=true)

### CoolProp Lists

Fluids list: http://coolprop.org/fluid_properties/PurePseudoPure.html#list-of-fluids

Properties list: http://www.coolprop.org/coolprop/HighLevelAPI.html#table-of-string-inputs-to-propssi-function

### Saturation Conditions

In [17]:
import CoolProp.CoolProp as cp  # for evaluating fluid thermophysical properties

# Saturation condition
fluid = 'Oxygen'  # http://www.coolprop.org/fluid_properties/PurePseudoPure.html
pressure = 2e5    # Pa, pressure

# Properties
density_liquid = cp.PropsSI('D', 'P', pressure, 'Q', 0, fluid)              # kg/m^3, saturated liquid density 
density_vapor = cp.PropsSI('D', 'P', pressure, 'Q', 1, fluid)               # kg/m^3, saturated vapor density
surf_tension = cp.PropsSI('surface_tension', 'P', pressure, 'Q', 0, fluid)  # N/m, saturated surface tension
temp_sat = cp.PropsSI('T', 'P', pressure, 'Q', 0, fluid)                    # K, saturation temperature

# Results
results = {"temp_sat": temp_sat, "density_liquid": density_liquid, "density_vapor": density_vapor, 
           "surf_tension": surf_tension}
print(results)

{'temp_sat': 97.23553533356629, 'density_liquid': 1105.401189711224, 'density_vapor': 8.354467802255993, 'surf_tension': 0.011405531150268775}


### Specific State Points

In [18]:
import CoolProp.CoolProp as cp    # for evaluating fluid thermophysical properties

# State points
fluid = 'Oxygen'  # http://www.coolprop.org/fluid_properties/PurePseudoPure.html
temp = 97         # K, temperature
pressure = 2e5    # Pa, pressure

# Properties
density = cp.PropsSI('D', 'T', temp, 'P', pressure, fluid)       # kg/m^3, fluid density
phase = cp.PhaseSI('T', temp, 'P', pressure, fluid)              # based on state point inputs
visc_dynamic = cp.PropsSI('V', 'T', temp, 'P', pressure, fluid)  # Pa-s, dynamic viscosity
prandtl = cp.PropsSI('Prandtl', 'T', temp, 'P', pressure, fluid)  # prandtl number

# Results
results = {"phase": phase, "density": density, "visc_dynamic": visc_dynamic, "prandtl": prandtl}
print(results)

{'phase': 'liquid', 'density': 1106.6351294355363, 'visc_dynamic': 0.0001638895690225786, 'prandtl': 2.004098847592044}


## Dimensionless Numbers

### Bond Number

In [19]:
import CoolProp.CoolProp as cp    # for evaluating fluid thermophysical properties

# Inputs
accel = 9.81e-6     # m/s^2, local acceleration at the interface
diam_fs = 6.6       # m, free surface diameter at the interface
pressure = 2e5      # Pa, saturation pressure at the interface
fluid = 'Oxygen'    # http://www.coolprop.org/fluid_properties/PurePseudoPure.html

# Properties
density_liquid = cp.PropsSI('D', 'P', pressure, 'Q', 0, fluid)              # kg/m^3, saturated liquid density
density_vapor = cp.PropsSI('D', 'P', pressure, 'Q', 1, fluid)               # kg/m^3, saturated vapor density
surf_tension = cp.PropsSI('surface_tension', 'P', pressure, 'Q', 0, fluid)  # N/m, saturated surface tension

# Function
def bond(accel, diam_fs, density_liquid, density_vapor, surf_tension):
    return (density_liquid - density_vapor) * accel * (diam_fs)**2 / surf_tension

# Results
print("Bond number =", round(bond(accel, diam_fs, density_liquid, density_vapor, surf_tension),1))

Bond number = 41.1


### Reynolds Number

In [20]:
import CoolProp.CoolProp as cp # for evaluating fluid thermophysical properties

# Inputs
length = 0.1        # m, characteristic length or hydraulic diameter = 4*area/wetted perimeter
velocity = 10       # m/s, fluid velocity
temp = 97           # K, bulk fluid temperature
pressure = 2e5      # Pa, dynamic pressure
fluid = 'Oxygen'    # http://www.coolprop.org/fluid_properties/PurePseudoPure.html

# Properties
density = cp.PropsSI('D', 'T', temp, 'P', pressure, fluid)         # kg/m^3, fluid density
visc_dynamic = cp.PropsSI('V', 'T', temp, 'P', pressure, fluid)    # Pa-s, dynamic viscosity

# Function
def reynolds(density, velocity, length, visc_dynamic):
    return density * velocity * length / visc_dynamic

# Results
print("Reynolds number =", "{:.5e}".format(reynolds(density, velocity, length, visc_dynamic)))

Reynolds number = 6.75232e+06


### Raleigh Number

### Nusselt Number

## Environments

# Passive CFM

## Tankage

## Venting

## Pressurization

## Chilldown and Filling

## Radiation Shields

# Active CFM

## Broad Area Cooling (BAC)

### Cooling Flow

For a given cooling load, the required flow in the BAC to provide that cooling can be estimated by:
* Identifying the cooling gas, its average operating pressure, and inlet temperature
* Specifying the cooling load needed
* Estimating the temperature drop from outlet to inlet of the BAC (e.g, based on past designs)

In [24]:
import CoolProp.CoolProp as cp # for evaluating fluid thermophysical properties

# Inputs
fluid = 'Helium'     # cooling fluid
cooling_load = 187.6 # W, cooling to be provided
pressure = 10e5      # Pa, average pressure of cooling fluid
temp_avg = 86.55     # K, average cooling fluid temperature
temp_delta = 3.1     # K, cooling fluid temperature difference (outlet - inlet)

 
def mdot_bac(fluid, cooling_load, pressure, temp_avg, temp_delta):
    """Mass flow rate of broad area cooling (BAC) loop in g/s"""
    # Specific heat capacity of the cooling gas, J/kg-K
    c_p = cp.PropsSI("C", "T", temp_avg, "P", pressure, fluid)
    # Mass flow rate, g/s
    massflow = cooling_load / c_p / temp_delta
    # Print results
    print("Required", fluid, "mass flow =", "{:.2e}".format(massflow), "kg/s")
    return

# Results
result = mdot_bac(fluid, cooling_load, pressure, temp_avg, temp_delta)


Required Helium mass flow = 1.16e-02 kg/s


### Thermal Performance

If the BAC loop is comprised of tubing mounted to a surface such as the tank wall or an intermediate shield, thermal performance can be estimated using a single stream heat exchanger analysis. The outlet temperature can then be calculated directly rather than assuming a value based on previous designs or experience. Note that this analysis assumes a constant temperature boundary on the tubing external wall.

In [12]:
import math
import CoolProp.CoolProp as cp
import CryoFM as cfm

# Inputs
fluid = 'Helium'    # cooling fluid
mdot_tube = 1.93e-4 # kg/s, cooling fluid mass flow per tube
id = 0.0127         # m, inside diameter of cooling tube
length = 3.556      # m, cooling tube length
temp_in = 85.0      # K, cooling fluid inlet temperature
temp_avg = 87.5     # K, average cooling fluid temperature
press_avg = 10e5    # Pa, average cooling fluid pressure
temp_wall = 90      # K, wall/shield temperature (assumed constant)

def hx_single_stream(fluid, mdot_tube, id, length, temp_in, temp_avg, 
                     press_avg, temp_wall):
    """Single stream heat exchanger with constant wall temperature
    
    Refr: Lienhard, A Heat Transfer Textbook, 4th ed., 2017
    """

    # Properties
    density = cp.PropsSI('D', 'T', temp_avg, 'P', press_avg, fluid)  # kg/m^3
    prandtl = cp.PropsSI('Prandtl', 'T', temp_avg, 'P', press_avg, fluid)
    conductivity = cp.PropsSI('conductivity', 'T', temp_avg, 'P', press_avg,
                              fluid) # W/m-K
    c_p = cp.PropsSI('C', 'T', temp_avg, 'P', press_avg, fluid) # J/kg-K


    # Flow area, perimeter, velocity, and Reynolds number
    area = math.pi / 4 * id**2  # m^2
    perimeter = math.pi * id
    velocity = mdot_tube / density / area  # m/s
    reynolds = cfm.reynolds(fluid, velocity, id, temp_avg, press_avg)

    # Friction factor and Nusselt number
    if reynolds < 2300:  # laminar
        flow = 'laminar'
        ffactor = 64 / reynolds
        nusselt = 3.657
    else:  # turbulent; 2300 < Re < 5e6
        flow = 'turbulent'
        ffactor = 1/((1.82 * math.log10(reynolds) - 1.64)**2)
        nusselt = (ffactor/8 * (reynolds-1000) * prandtl / 
                  (1 + 12.7*math.sqrt(ffactor/8) * (prandtl**(2/3) - 1)))

    # Corrected friction factor and Nusselt for variable fluid properties
    tratio = temp_avg / temp_wall
    f_corr = ffactor * (tratio)**0.23
    if tratio < 0.14 or tratio > 3.3: # caution message
        print("Caution: fluid-to-wall temperature ratio out of range for \
              corrected friction factor correlation used")
    if temp_wall > temp_avg:
        n = 0.47
    else:
        n = 0
    nu_corr = nusselt * (tratio)**n

    # Heat transfer coefficient and outlet temperature
    ht_coeff = nu_corr * conductivity / id  # W/m^2-K
    temp_out = ((temp_wall - temp_in) * 
                (1 - math.exp(-ht_coeff*perimeter*length/mdot_tube/c_p))
                + temp_in)

    # Print selected results
    if reynolds > 5e6: # out of range caution
        print("Caution: Reynolds number above stated range for Nusselt \
              correlation used")
    print("Properties: density =", round(density,2), "kg/m^3, Prandtl =", 
          round (prandtl,3), ", conductivity =", "{:.3e}".format(conductivity),
          "W/m-K, Cp =", round(c_p,2), "J/kg-K\n")
    print("Flow: area =", "{:.3e}".format(area), "m^2, velocity =", 
          round(velocity,2), "m/s, perimeter =", "{:.3e}".format(perimeter), 
          "m, Reynolds =", "{:.3e}".format(reynolds), flow, end='\n\n')
    print("Other parameters: friction factor (corrected) =", 
          "{:.3e}".format(f_corr), ", Nusselt number (corrected) =", 
          "{:.3e}".format(nu_corr), end='\n\n')
    print("Results: heat transfer coefficient:", round(ht_coeff,2), 
          "W/m^2-K, outlet temp =", round(temp_out,2))
    return

# Call function
hx_single_stream(fluid, mdot_tube, id, length, temp_in, temp_avg, press_avg, 
                 temp_wall)

Properties: density = 5.42 kg/m^3, Prandtl = 0.696 , conductivity = 6.836e-02 W/m-K, Cp = 5216.74 J/kg-K

Flow: area = 1.267e-04 m^2, velocity = 0.28 m/s, perimeter = 3.990e-02 m, Reynolds = 2.123e+03 laminar

Other parameters: friction factor (corrected) = 2.996e-02 , Nusselt number (corrected) = 3.609e+00

Results: heat transfer coefficient: 19.43 W/m^2-K, outlet temp = 89.68


### Wall or Shield dT

### Manifolds Sizing

### Pressure Drop

## Mixing

# Supplemental

## Verification Tests

In [23]:
if(abs(bond(9.81E-6, 6.6, 1105.401189711224, 8.354467802255993, 0.011405531150268775) - 41.1)) > 0.1:
    print("FAILED: bond function verification case")
else:
    print("PASSED: bond function verification case")

if(abs(reynolds(1106.6351294355363, 10, 0.1, 0.0001638895690225786) - 6.752322E6)) > 0.1:
    print("FAILED: reynolds function verification case")
else:
    print("PASSED: reynolds function verification case")
        

PASSED: bond function verification case


TypeError: 'float' object is not callable

## Binder

Setting it up from GitHub: https://github.com/Build-a-binder/build-a-binder.github.io/blob/master/workshop/10-zero-to-binder.md

## Pint

website: https://pint.readthedocs.io/en/stable/


In [ ]:
import pint

# Mixed units math
units = pint.UnitRegistry()
sum = 3 * units.m + 4 * units.cm
print(sum)

# Unit conversions
length = 1 * units.m
print(length.to(units.ft))

# Specify value and units    
value = units.Quantity
length = value('2.54 cm')
print(length)

## Code Snippets

In [ ]:
# Full printing of inputs and properties from bond and raleigh functions
print("Inputs: fluid =", fluid, ", pressure =", "{:.3e}".format(pressure), "Pa", ", free surface diameter =",
      round(diam_fs, 3), "m", ", acceleration =", "{:.3e}".format(accel), "m/s^2", ", liquid density =", 
      round(density_liquid, 1), "kg/m^3", ", vapor density =", round(density_vapor,4), "kg/m^3", 
      ", surface tension =", "{:.5e}".format(surf_tension), "N/m", end='\n\n')
print("Inputs: fluid =", fluid, ", pressure =", "{:.3e}".format(pressure), "Pa", ", fluid temperature =",
      round(temp, 2), "K", ", characteristic dimension =", round(length, 3), "m", ", fluid velocity =", 
      round(velocity, 3), "m/s", ", fluid density =", "{:.5e}".format(density), "kg/m^3",
      ", dynamic viscosity =", "{:.5e}".format(visc_dynamic), "Pa-s", end='\n\n')